In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import poisson

def dI_dt(I, E, lambda_E, lambda_R):
    return lambda_E * E - lambda_R * I

def dE_dt(S, E, lambda_S, f_q):
    return lambda_S * S * f_q - lambda_E * E

def dS_dt(S, I, lambda_S, lambda_R, f_q):
    return -lambda_S * S * f_q + lambda_R * I

def f_q(K, M):
    return K / (M + K)

def simulate_virus_spread(S0, E0, I0, lambda_S, lambda_E, lambda_R, M, total_time, dt, alpha):
    num_steps = int(total_time / dt) + 1
    t = np.linspace(0, total_time, num_steps)
    S = np.zeros(num_steps)
    E = np.zeros(num_steps)
    I = np.zeros(num_steps)
    total_population = np.zeros(num_steps)

    # Initial conditions
    S[0], E[0], I[0] = S0, E0, I0

    # Simulation loop
    for i in range(1, num_steps):
        q_t = f_q(I[i-1], M)  # Calculate q(t) based on the current number of infected items
        lambda_t = alpha * q_t  # Calculate lambda(t) based on q(t)

        # Generate the number of new infections in the time interval dt using Poisson distribution
        new_infections = poisson.rvs(lambda_t * dt)

        # Update the populations using the differential equations
        dS = dS_dt(S[i-1], I[i-1], lambda_S, lambda_R, q_t)
        dE = dE_dt(S[i-1], E[i-1], lambda_S, q_t)
        dI = dI_dt(I[i-1], E[i-1], lambda_E, lambda_R) + new_infections

        S[i] = S[i-1] + dS * dt
        E[i] = E[i-1] + dE * dt
        I[i] = I[i-1] + dI * dt

        total_population[i] = S[i] + E[i] + I[i]

    return t, S, E, I, total_population

# Parameters for simulation
S0 = 1000   # Initial number of susceptible peers
E0 = 0     # Initial number of exposed peers
I0 = 1     # Initial number of infected peers
lambda_S = 0.1  # Rate of new file downloads per minute
lambda_E = 0.001  # Rate of becoming infected per minute
lambda_R = 0.0001 # Rate of recovery per minute
M = 1000         # Number of uninfected items
total_time = 1200   # Total simulation time (in minutes)
dt = 0.1        # Time step for integration
alpha = 0.1     # Scaling factor for lambda(t)

# Simulate virus spread
t, S, E, I, total_population = simulate_virus_spread(S0, E0, I0, lambda_S, lambda_E, lambda_R, M, total_time, dt, alpha)

# Plotting using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=S, mode='lines', name='Susceptible'))
fig.add_trace(go.Scatter(x=t, y=E, mode='lines', name='Exposed'))
fig.add_trace(go.Scatter(x=t, y=I, mode='lines', name='Infected'))

fig.update_layout(
    title='Simulation of Virus Spread with Poisson Distribution',
    xaxis_title='Time (minutes)',
    yaxis_title='Population',
    template='plotly_white'
)

fig.show()


